In [1]:
from autogen_agentchat.agents import AssistantAgent

In [8]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import  load_dotenv
load_dotenv()
import os

api_key=os.getenv("OPENAI_API_KEY")


In [10]:

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=api_key)

agent= AssistantAgent(model_client=model_client,name="AssistantAgent", description="An assistant agent that can answer questions and provide information.")

import asyncio

message = "What is the capital of France?"

response= await agent.run(task=message)

print(f"Response: {response}")

Response: messages=[TextMessage(id='05a0e608-32c6-4b94-94f3-99c0221fac64', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 12, 52, 23, 445213, tzinfo=datetime.timezone.utc), content='What is the capital of France?', type='TextMessage'), TextMessage(id='3a687217-276f-449a-ab57-58546a6856e6', source='AssistantAgent', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 7, 13, 12, 52, 25, 149864, tzinfo=datetime.timezone.utc), content='The capital of France is Paris.', type='TextMessage')] stop_reason=None


/var/folders/lm/bycxy6bd2rdgh0j_j17ty8pw0000gn/T/ipykernel_2377/3567253170.py:9: RuntimeWarning: coroutine 'BaseChatAgent.run' was never awaited
  response= await agent.run(task=message)


In [12]:
result =await agent.run(task="What question I asked you earlier ?")
print(f"Result: {result}")

Result: messages=[TextMessage(id='e538868d-c459-44d7-a4a5-20d10b19cf44', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 12, 53, 38, 17222, tzinfo=datetime.timezone.utc), content='What question I asked you earlier ?', type='TextMessage'), TextMessage(id='1c6190e6-362a-47fe-9924-0d8d0426e3fd', source='AssistantAgent', models_usage=RequestUsage(prompt_tokens=93, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 7, 13, 12, 53, 39, 376488, tzinfo=datetime.timezone.utc), content='You asked, "What question I asked you earlier?"', type='TextMessage')] stop_reason=None


In [14]:
agent_state = await agent.save_state()
print(agent_state)


{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'What is the capital of France?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'The capital of France is Paris.', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}, {'content': 'What question I asked you earlier ?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'You asked, "What is the capital of France?"', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}, {'content': 'What question I asked you earlier ?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'You asked, "What question I asked you earlier?"', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}]}}


In [ ]:
{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'What is the capital of France?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'The capital of France is Paris.', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}, {'content': 'What question I asked you earlier ?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'You asked, "What is the capital of France?"', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}, {'content': 'What question I asked you earlier ?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'You asked, "What question I asked you earlier?"', 'thought': None, 'source': 'AssistantAgent', 'type': 'AssistantMessage'}]}}

In [15]:
from pydantic import BaseModel
class PlanetInfo(BaseModel):
    name:str
    color:str
    distance:str
structured_model_client = OpenAIChatCompletionClient(
    model='gpt-4o',
    api_key=api_key,
    response_format=PlanetInfo
)


my_assistant_2 = AssistantAgent(name='Assistant2',model_client=structured_model_client)
result = await my_assistant_2.run(task='Who are you')
print(result)

messages=[TextMessage(id='9a4175c7-ca12-49c2-a786-4691f35822f3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 17, 39, 24, 24590, tzinfo=datetime.timezone.utc), content='Who are you', type='TextMessage'), TextMessage(id='a9f1dd61-1a22-4916-9d24-bfe301237384', source='Assistant2', models_usage=RequestUsage(prompt_tokens=96, completion_tokens=32), metadata={}, created_at=datetime.datetime(2025, 7, 13, 17, 39, 26, 694379, tzinfo=datetime.timezone.utc), content='{"name":"Pluto","color":"Red-brown","distance":"About 5.9 billion kilometers (3.67 billion miles) from the Sun"}', type='TextMessage')] stop_reason=None


In [10]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")


model_client=OpenAIChatCompletionClient(model='gpt-4o',api_key=api_key)

import math

def calculate_emi_breakdown(principal_amount: float, annual_interest_rate: float, years: int) -> str:
    """
    Calculates the Equated Monthly Instalment (EMI) and the breakdown of the first month's payment.

    Args:
        principal_amount (float): The total loan amount.
        annual_interest_rate (float): The annual interest rate (e.g., 5 for 5%).
        years (int): The loan term in years.

    Returns:
        str: A formatted string detailing the monthly EMI, and the principal and interest
             components for the first month.
    """
    # Convert annual rate to monthly rate and percentage to decimal
    monthly_rate = (annual_interest_rate / 12) / 100
    # Convert years to months
    months = years * 12

    # Handle the edge case of 0% interest
    if monthly_rate == 0:
        emi = principal_amount / months
        interest_first_month = 0.0
    else:
        # Calculate EMI using the formula
        emi = (principal_amount * monthly_rate * (1 + monthly_rate)**months) / ((1 + monthly_rate)**months - 1)
        # Calculate the interest component for the first month
        interest_first_month = principal_amount * monthly_rate

    # Calculate the principal component for the first month
    principal_first_month = emi - interest_first_month

    # Return a nicely formatted string
    return (
        f"Total Monthly EMI: ${emi:,.2f}\n"
        f"For the first month's payment:\n"
        f"  - Principal Component: ${principal_first_month:,.2f}\n"
        f"  - Interest Component: ${interest_first_month:,.2f}"
    )

emi_tool = FunctionTool(calculate_emi_breakdown,description='A tool to calculate an EMI for prinicipal amount,Years, Interest rate that user provides')



agent = AssistantAgent(
    name="ReverseStringAgent",
    model_client= model_client,
    system_message='You are a helpful assistant that can calculate an EMI for prinicpal amount,Years, Interest rate that user provides by using the tool calculate_emi_breakdown. The response from the tool is the Breakdown of EMI. Give the result with summary',
    tools=[emi_tool]
)


result = await agent.run(task = 'Calculate EMI for a loan of $52000 at 9% annual interest over 10 years')
print(result.messages[-1].content)

Total Monthly EMI: $658.71
For the first month's payment:
  - Principal Component: $268.71
  - Interest Component: $390.00


In [11]:
print(result.messages)


[TextMessage(id='e5ac9fb2-5807-46b6-bd38-fa6e41799c94', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 6, 28, 56, 325320, tzinfo=datetime.timezone.utc), content='Calculate EMI for a loan of $52000 at 9% annual interest over 10 years', type='TextMessage'), ToolCallRequestEvent(id='a4911f31-8b62-4606-80fb-772563ffc003', source='ReverseStringAgent', models_usage=RequestUsage(prompt_tokens=163, completion_tokens=29), metadata={}, created_at=datetime.datetime(2025, 7, 16, 6, 28, 57, 340274, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_5ZFR59FoZyfC2KKCFvjmNSe3', arguments='{"principal_amount":52000,"annual_interest_rate":9,"years":10}', name='calculate_emi_breakdown')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='a94360bd-00c5-49da-9b04-8b8526860410', source='ReverseStringAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 16, 6, 28, 57, 341102, tzinfo=datetime.timezone.utc), content=[FunctionEx